In [1]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_3164/945712042.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
import pandas as pd
import os
import argparse
from typing import Any, Callable, Sequence
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.flatten_util import ravel_pytree
import numpy as np
import matplotlib.pyplot as plt
from flax import linen as nn


from bong.util import run_rebayes_algorithm, gaussian_kl_div, MLP
from bong.src import bbb, blr, bog, bong, experiment_utils
from bong.agents import AGENT_NAMES
from plot_utils import *



In [20]:
from plot_utils import *

root_dir = '/teamspace/studios/this_studio/jobs'
data_dir = 'reg-D10-mlp_20_20_1'
model_dir = 'mlp_10_10_1'
agent_dir = 'A:Any-P:Any-Lin:1-LR:Any-IT:10-MC:10-EF:1-R:10'
dir = f'{root_dir}/{data_dir}/{model_dir}/{agent_dir}'





In [25]:

def extract_metrics_from_files(dir, exclude_val=True):
    fname = f"{dir}/jobs.csv"
    df = pd.read_csv(fname)
    jobnames = df['jobname']
    jobname = jobnames[0]
    fname = f"{dir}/{jobname}/work/results.csv"
    df_res = pd.read_csv(fname)
    metrics =  df_res.columns
    if exclude_val:
        metrics = [m for m in metrics if "_val" not in m ]
    return metrics

metrics = extract_metrics_from_files(dir)
print(metrics)

['nll', 'nlpd']


In [26]:
import datetime
import pytz

# Get the current time in UTC
now_utc = datetime.datetime.now(pytz.utc)

# Convert the current time to PST
pst_timezone = pytz.timezone('US/Pacific')
now_pst = now_utc.astimezone(pst_timezone)

# Format the datetime string in PST
datetime_string = now_pst.strftime("%Y-%m-%d:%H-%M-%S")

print(datetime_string)


2024-05-14:20-23-56
